## Retrieving and Indexing text into Pinecone (external_knowledge_Ingredient.csv)

### Option 1: use gemini ai - GoogleGenerativeAIEmbeddings for embedding model (free model from gemini: models/embedding-001) - SUCCESS

In [ ]:
!pip install -qU langchain langchain-community langchain-cohere langchain-pinecone langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB

In [ ]:
!pip install --upgrade langchain-pinecone

### PHASE-1

In [ ]:
import pandas as pd
import numpy as np
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone as LangChainPinecone
from pinecone import Pinecone, ServerlessSpec
import os
from typing import List, Dict
import json
import getpass
import os

In [ ]:
# Pinecone AUTH
api_key_pinecone = getpass.getpass("Input API KEY PINECONE CLOUD")
print('api_key_pinecone telah diinput')

Input API KEY PINECONE CLOUD··········
api_key_pinecone telah diinput


Load CSV into Pandas

In [ ]:
# Load your file (assuming you uploaded it manually or via drive)
df_ingredient = pd.read_csv("/content/External_knowledge_Ingredient.csv")
df_ingredient = df_ingredient.dropna()
# df = df.iloc[:10,]
print(df_ingredient.shape)
# print(df["__sheet__"].value_counts())
df_ingredient.head()

(100, 2)


,name,quantity
0,Chicken,1.2 kg
1,Onion,5 thinly sliced
2,Tomatoes,2 finely chopped
3,Garlic,8 cloves chopped
4,Ginger paste,1 tbsp


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Prepare Data for Embedding

In [ ]:
# Add sheet label
df_ingredient["__sheet__"] = "Ingredient"
df_ingredient.head()

,name,quantity,__sheet__
0,Chicken,1.2 kg,Ingredient
1,Onion,5 thinly sliced,Ingredient
2,Tomatoes,2 finely chopped,Ingredient
3,Garlic,8 cloves chopped,Ingredient
4,Ginger paste,1 tbsp,Ingredient


Convert Rows to Text Documents for Embedding

In [ ]:
def row_to_text(row):
    sheet = str(row.get("__sheet__", "")).strip()

    if sheet == "NutritionInfo":
        return (
            f"Nutrition facts for {row.get('name', 'unknown')}: "
            f"{row.get('calories', 'N/A')} calories, "
            f"{row.get('protein_g', 'N/A')}g protein, "
            f"{row.get('carbohydrates_total_g', 'N/A')}g carbohydrates, "
            f"{row.get('fat_total_g', 'N/A')}g fat, "
            f"{row.get('fiber_g', 'N/A')}g fiber."
        )

    elif sheet == "Ingredient":
        return (
            f"Ingredient: {row.get('name', 'unknown')}. "
            f"Quantity: {row.get('quantity', 'unspecified')}. "
        )

    elif sheet == "Recipe":
        return (
            f"Recipe Title: {row.get('title', 'Untitled')}. "
            f"Description: {row.get('description', '')}. "
            f"Ingredients: {row.get('ingredients', '')}. "
            f"Instructions: {row.get('instructions', '')}. "
            f"Nutrition Score: {row.get('nutrition', 'unknown')}. "
        )

    return "Uncategorized entry."

# Apply to all rows
texts = df_ingredient.apply(row_to_text, axis=1).tolist()
metadatas = df_ingredient.to_dict(orient="records")

print(f"Converted {len(texts)} rows to documents.")

Converted 100 rows to documents.


Setup Gemini AI Embedding Model

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
# Gemini AI AUTH
import getpass
import os

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

Chunk the Documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = []
chunk_metadata = []

for i, text in enumerate(texts):
    splits = text_splitter.split_text(text)
    chunks.extend(splits)
    chunk_metadata.extend([metadatas[i]] * len(splits))

Embedding and Store to Pinecone

In [ ]:
from langchain.vectorstores import Pinecone  # <--- THIS is LangChain's wrapper
import pinecone  # This is the official SDK
from time import sleep

In [ ]:
embeddings = embedding_model.embed_documents(texts)
print(f"Generated {len(embeddings)} embeddings.")

Generated 100 embeddings.


In [ ]:
# Initialize Pinecone
pinecone_api_key = "API KEY PINECONE"
pc = pinecone.Pinecone(api_key=pinecone_api_key)

index_name = "nutrition-rag-index"

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# index_name = 'nutrition-rag-index'
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# Create vector index if not exist
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=spec
    )
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# Connect to index
index = pc.Index(index_name)

Format Vectors + Metadata

In [ ]:
# Each entry must be a tuple of (id, vector, metadata)
vectors = [
    {
        "id": f"vec-{i}",
        "values": embeddings[i],
        "metadata": chunk_metadata[i]
    }
    for i in range(len(embeddings))
]

In [ ]:
# Batch upsert (up to 100 vectors per call is safe)
for i in range(0, len(vectors), 100):
    batch = vectors[i:i+100]
    index.upsert(vectors=batch)

print("Vector embeddings created successfully!")
print(f"Stored {len(texts)} document chunks with enhanced metadata")

Vector embeddings created successfully!
Stored 100 document chunks with enhanced metadata


### PHASE-2

Search with LangChain Retriever

In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# LangChain wrapper around Pinecone index
vectorstore = Pinecone(
    index=index,               # from pinecone.Index
    embedding=embedding_model
)

Create a Retriever (with optional filter)

In [ ]:
# Create retriever with filter (e.g. meal_type = 'lunch')
retriever = vectorstore.as_retriever(
    search_kwargs={
        "filter": {"meal_type": "lunch"},
        "k": 5  # top 5 similar results
    }
)

Test the Retriever with a Sample Query

In [ ]:
query = "high protein salad with chicken"
docs = retriever.get_relevant_documents(query)

# Display results
for i, doc in enumerate(docs):
    print(f"Result {i+1}")
    print("Title:", doc.metadata.get("title"))
    print("Meal Type:", doc.metadata.get("meal_type"))
    print("Content:", doc.page_content[:300], "...")
    print("-" * 50)


RAG QA Chain Sample

In [ ]:
from langchain.chat_models import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             temperature=0,
                             max_tokens=None,
                             timeout=None,
                             temperature=0.3)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

response = qa_chain.run("Give me a lunch recipe that's low in carbs but high in protein.")
print("Answer:", response)

### Option 2: use cohere - CohereEmbeddings for embedding model (paid model from cohere: models/embed-english-v3.0, but it's rate-limited and not intended for production use)